In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as tfhub
import sqlite3
from sqlite3 import Error

pd.set_option('display.max_colwidth', -1)

In [2]:
conn = sqlite3.connect('wine_data.sqlite')
c = conn.cursor()

In [22]:
wine_df = pd.read_sql('Select * from wine_data', conn)

In [24]:
wine_df.shape

(100228, 10)

In [5]:
embed = tfhub.Module("C:/Users/Administrator/Downloads/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47")

In [9]:
#download the model to local so it can be used again and again
#!mkdir ../wine_model/module_useT
# Download the module, and uncompress it to the destination folder. 
#!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ../wine_model/module_useT

The syntax of the command is incorrect.


In [6]:
    
wd = list(wine_df.description)
output = embed(wd)


In [7]:
#embed = tfhub.Module("C:/Users/Administrator/Downloads/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47")

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    wine_embeddings = session.run(output)
    print(wine_embeddings.shape)

(10, 512)


In [18]:
    
g=tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype = tf.string, shape=[None])
    embed = tfhub.Module("C:/Users/Administrator/Downloads/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47")
    em_txt = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
#g.finalize()

session = tf.Session(graph = g)
session.run(init_op)


In [25]:
result = session.run(em_txt, feed_dict={text_input:list(wine_df.description)})


In [26]:
def recommend_engine(query, embedding_table = result):
    # Embed user query
    with tf.Session(graph = g) as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embedding = session.run(embed([query]))

    # Calculate similarity with all reviews
    similarity_score = np.dot(embedding, embedding_table.T)
    
    # Recommend
    recommendations = wine_df.copy()
    recommendations['recommendation'] = similarity_score.T
    recommendations = recommendations.sort_values('recommendation', ascending=False)
    recommendations = recommendations[['variety', 'title', 'price', 'description', 'recommendation', 'rating']]

    return recommendations

In [27]:
query = "fruity, rich, easy to drink, sweet"

recommendation = recommend_engine(query)
print(query)

recommendation.head().T

fruity, rich, easy to drink, sweet


,28652,85022,83116,48449,87765
variety,Syrah,Cabernet Sauvignon,Cabernet Sauvignon,Cabernet Sauvignon,Cabernet Sauvignon
title,Dierberg 2008 Syrah (Santa Ynez Valley),Middle Sister NV Mischief Maker Cabernet Sauvignon (California),Kirkland Signature 2009 Cabernet Sauvignon (Alexander Valley),Wattle Creek 2010 Cabernet Sauvignon (Alexander Valley),Heron 2009 Cabernet Sauvignon (Mendocino)
price,34,12,9,42,13
description,"A nice Syrah, rich in berries, licorice, Dr. Pepper cola and spices, with soft, luxurious tannins. It's a little uncomplicated and sweet, but easy to drink now.","With an aroma like cherry cola, and fruity, almost sweet flavors, this medium-bodied wine is easy to drink.","Fruity in blackberry and cherry flavors, this is easy to drink, with a good acid-tannin balance.","Lots of flashy blackberry and cassis flavors in this Cabernet. Easy to drink, but feels way too soft to earn a higher score. Drink up.","A simple Cabernet, the kind to drink at a party and not fuss. It's dry and smooth, with berry and herb flavors."
recommendation,0.808436,0.805091,0.782239,0.780206,0.776853
rating,86,85,84,84,84
